In [ ]:
import mlflow
from mlflow.models.signature import infer_signature
import xgboost as xgb
import numpy as np

In [ ]:
os.environ['MLFLOW_TRACKING_TOKEN'] = '' #set token
os.environ['MLFLOW_TRACKING_URI'] = "https://mlflow-demo.software-dev.ncsa.illinois.edu"

mlflow.set_tracking_uri('https://mlflow-demo.software-dev.ncsa.illinois.edu') 
mlflow.set_experiment("test-experiment")

In [ ]:
train_data = np.concatenate((np.random.randn(10000,10), 2*np.random.randn(10000,10)+2))
train_labels = np.concatenate((np.ones(10000), np.zeros(10000)))

test_data = np.concatenate((np.random.randn(100,10), 2*np.random.randn(100,10)+2))
test_labels = np.concatenate((np.ones(100), np.zeros(100)))

In [ ]:
params = {
    'reg_lambda': 1,
    'reg_alpha': 0.5,
    'n_estimators': 5,
    'min_child_weight': 379,
    'max_depth': 10,
    'learning_rate': 0.5,
}

In [ ]:
with mlflow.start_run(run_name="MY_RUN") as run:
    
    # log params
    for param_name, value in params.items():
        mlflow.log_param(param_name, value)

    # train model
    model = xgb.XGBClassifier(nthread=-1, **params)
    model.fit(train_data, train_labels)
    
    # log metrics
    mlflow.log_metric('train_score', model.score(train_data, train_labels))
    mlflow.log_metric('test_score', model.score(test_data, test_labels))
    
    # log model
    signature = infer_signature(test_data, model.predict(test_data))
    mlflow.xgboost.log_model(model, "model", signature=signature)